In [22]:
import torch 
from torch.utils.data import Dataset, ConcatDataset
from tqdm import tqdm
import os 
import dill as pickle


In [23]:
raw_data_dir = '/home/kuba/Documents/Data/Raw/RatEEG_R/'     #THIS NEEDS TO BE ABSOLUTE PATH
train_percent = 0.7
test_percent = 1 - train_percent
path_to_save_dir = '/home/kuba/Documents/Data/Datasets/RatEEG_D/DS01' #path to save the data too


In [24]:
class CombinedDataSet(Dataset):
    def __init__(self, raw_data_path):
        X, y = torch.load(raw_data_path)
        self.x = X
        self.y = y
        self.n_samples = len(self.x)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [25]:
def save_dataset(dataset, path_to_save_dir, file_name):
    with open(os.path.join(path_to_save_dir,file_name), 'wb') as f:
        pickle.dump(dataset, f)

In [26]:
all_datasets = []
for file_name in tqdm(sorted(os.listdir(raw_data_dir))):
    if file_name.endswith('.pt'):
        raw_data_path = os.path.join(raw_data_dir, file_name)
        dataset = CombinedDataSet(raw_data_path)
        all_datasets.append(dataset)

combined_dataset = ConcatDataset(all_datasets)

100%|██████████| 33/33 [00:00<00:00, 85.87it/s]


In [27]:
# split into train and test
total_size = len(combined_dataset)
train_size = int(train_percent * total_size)
test_size = total_size - train_size

train_dataset, test_dataset = torch.utils.data.random_split(combined_dataset, [train_size, test_size])

In [28]:
#save to non raw data to data Datasets

os.makedirs(path_to_save_dir, exist_ok=False)

save_dataset(train_dataset, path_to_save_dir, 'combined_train.pkl')
save_dataset(test_dataset, path_to_save_dir, 'combined_test.pkl')

FileExistsError: [Errno 17] File exists: '/home/kuba/Documents/Data/Datasets/RatEEG_D/DS01'